In [11]:
import pandas as pd
data_pol=pd.read_csv('C:/Users/Usuario/Documents/Men_argue-Nature_acts/CSV/pollution_pop.csv')
data_city=pd.read_csv('C:/Users/Usuario/Documents/Men_argue-Nature_acts/CSV/cities_database.csv')

In [12]:

data_pol.columns = data_pol.columns.str.strip()

columns_to_convert = ['pm25', 'pm10', 'o3', 'no2', 'so2', 'co', 'year', 'pop']
data_pol[columns_to_convert] = data_pol[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [13]:

columns_to_fill = ['pm25', 'pm10', 'o3', 'no2', 'so2']

for column in columns_to_fill:
    mean_value = data_pol[column].mean()
    data_pol[column].fillna(mean_value, inplace=True)

In [14]:
data_pol['co'] = data_pol['co'].fillna(0)

In [15]:
data_pol.columns

Index(['pm25', 'pm10', 'o3', 'no2', 'so2', 'co', 'year', 'pop'], dtype='object')

In [16]:
def get_city_data(madrid):
    import requests
    import pandas as pd
    import json
    
    api_url = 'https://api.api-ninjas.com/v1/city?name={}'.format(city_name)
    response = requests.get(api_url, headers={'X-Api-Key': 'XhKog+U7kqeVm1jW7MeNDg==ZsJmH2sbNbEnRq3H'})
    if response.status_code == requests.codes.ok:
        response_text = response.text
        data = json.loads(response_text)
        df = pd.DataFrame(data)
        return df
    else:
        print("Error:", response.status_code, respoonse_text)
        return None
city_name = 'madrid'
city_df = get_city_data(city_name)
if city_df is not None:
    print(city_df)

     name  latitude  longitude country  population  is_capital
0  Madrid   40.4189    -3.6919      ES     3266126        True


In [17]:
city_df.drop(columns='population',axis=1,inplace=True)


In [18]:
data_pol = data_pol.assign(key=1)
city_df = city_df.assign(key=1)



In [19]:
result = pd.merge(city_df, data_pol, on='key').drop('key', axis=1)

In [20]:
result = result.rename(columns={'pop': 'population', 'pm25': 'pm2.5'})

In [21]:
new_order = ['name', 'latitude', 'longitude', 'country', 'population', 'is_capital', 'co', 'no2', 'o3', 'so2', 'pm2.5', 'pm10', 'overall_aqi']
result = result.reindex(columns=new_order)

In [22]:
result['overall_aqi'].fillna(0,inplace=True)

In [23]:
result

,name,latitude,longitude,country,population,is_capital,co,no2,o3,so2,pm2.5,pm10,overall_aqi
0,Madrid,40.4189,-3.6919,ES,6044000,True,0.0,18.0,20.0,2.000000,55.805431,13.0,0.0
1,Madrid,40.4189,-3.6919,ES,6132000,True,0.0,26.0,60.0,2.000000,73.000000,39.0,0.0
2,Madrid,40.4189,-3.6919,ES,6132000,True,0.0,12.0,42.0,2.000000,69.000000,21.0,0.0
3,Madrid,40.4189,-3.6919,ES,6132000,True,0.0,17.0,58.0,2.000000,36.000000,21.0,0.0
4,Madrid,40.4189,-3.6919,ES,6132000,True,0.0,26.0,53.0,3.000000,40.000000,46.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,Madrid,40.4189,-3.6919,ES,6783000,True,0.0,11.0,28.0,3.105897,43.000000,9.0,0.0
3590,Madrid,40.4189,-3.6919,ES,6783000,True,0.0,22.0,19.0,3.105897,45.000000,13.0,0.0
3591,Madrid,40.4189,-3.6919,ES,6783000,True,0.0,20.0,17.0,3.105897,62.000000,16.0,0.0
3592,Madrid,40.4189,-3.6919,ES,6783000,True,0.0,11.0,31.0,3.105897,41.000000,13.0,0.0


In [24]:
-----------------------------------------------------------------------------

SyntaxError: invalid syntax (162911398.py, line 1)

In [ ]:
from sklearn.model_selection import train_test_split

# Define the features (X) and the target variables (y)
features = result.drop(['population','name','latitude','longitude','country','is_capital'], axis=1)  # All columns except the target variables
target = result[['co','pm2.5', 'pm10', 'o3', 'no2', 'so2']]  # Target variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the training features
X_train_normalized = scaler.fit_transform(X_train)

# Transform the testing features
X_test_normalized = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the Random Forest Regression model
rfr_model_normalized = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the normalized training data
rfr_model_normalized.fit(X_train_normalized, y_train)

# Make predictions using the trained model on the normalized testing data
y_pred_normalized = rfr_model_normalized.predict(X_test_normalized)

# Evaluate the model using Mean Squared Error (MSE)
mse_normalized = mean_squared_error(y_test, y_pred_normalized)
print('Mean Squared Error (Normalized Data):', mse_normalized)

Mean Squared Error (Normalized Data): 5.63714616659931


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the Random Forest Regression model
rfr_model = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust the n_estimators parameter as needed

# Train the model on the training data
rfr_model.fit(X_train, y_train)

# Make predictions using the trained model
y_pred = rfr_model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 4.734896122647636


In [ ]:
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (if not already split)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize Lazy Regressor
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit and score all models
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

100%|██████████| 42/42 [00:10<00:00,  3.85it/s]


In [ ]:
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (if not already split)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize Lazy Regressor
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit and score all models
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

100%|██████████| 42/42 [00:12<00:00,  3.33it/s]


In [ ]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import joblib

joblib.dump(rfr_model, 'pred_model.joblib')

['pred_model.joblib']

In [ ]:
pip install --upgrade scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/7b/0e/25d6b5678ed3c7e12bc94d047d0e9492e89cc78b7ea0034ac0f1cf2ff304/scikit_learn-1.4.1.post1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.6 MB 1.3 MB/s eta 0:00:09
   --- ------------------------------------ 0.9/10.6 MB 9.5 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/10.6 MB 21.3 MB/s eta 0:00:01
   ---------------------- ----------------- 6.0/10.6 MB 29.4 MB/s eta 0:00:01
   --------------------------------- ------ 9.0/10.6 MB 36.1 MB/s eta 0:00:01
   ---------------------------------------  10.6/10.6 MB 50.1 MB/s eta 0:00:01
   ---------------------------------------  10.6/10.6 MB 50.1 MB/s eta 0:00:01
   ---------------------------------------- 10.6/10.6 MB 36.2 MB/s eta 0:00:00
  Attemptin

ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\Usuario\\anaconda3\\Lib\\site-packages\\~klearn\\utils\\murmurhash.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
import sklearn; print(sklearn.__version__)

1.3.0
